# 5.2 小数据集训练CNN-其他对照组（dropout/L1/L2等其他抑制过拟合技术）
## 基准组实验条件
### 数据集：小数据集
### 训练集：1,000/类别
### 验证集：500/类别
### 测试集：500/类别
### 是否使用数据增强：否/是
### 训练轮数：30
### 可参考教材pp111-115

## 1 对照组之一：仅使用dropout选项

### 1.1 按需建模，dropout = 0.5

In [56]:
from keras import models
from keras import layers

#conv2D的filters从32增加到128
my_network_dropout = models.Sequential()
#32个filter
my_network_dropout.add(layers.Conv2D(32,(3,3),activation = 'relu',input_shape = (150,150,3)))
my_network_dropout.add(layers.MaxPooling2D((2,2)))
#64个filter
my_network_dropout.add(layers.Conv2D(64,(3,3),activation = 'relu'))
my_network_dropout.add(layers.MaxPooling2D((2,2)))
#128个filter
my_network_dropout.add(layers.Conv2D(128,(3,3),activation = 'relu'))
my_network_dropout.add(layers.MaxPooling2D((2,2)))
#128个filter
my_network_dropout.add(layers.Conv2D(128,(3,3),activation = 'relu'))
my_network_dropout.add(layers.MaxPooling2D((2,2)))

my_network_dropout.add(layers.Flatten()) #将3D展平为1D
#为防止过拟合，使用dropout
my_network_dropout.add(layers.Dropout(0.5))
my_network_dropout.add(layers.Dense(512, activation = 'relu'))
my_network_dropout.add(layers.Dense(1,activation='sigmoid'))#最后输出二元分类概率


from keras import optimizers
my_network_dropout.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['acc'])

### 1.2 训练模型

In [57]:
history_droupout = my_network_dropout.fit_generator(
    train_generator,
    steps_per_epoch = 100,
    epochs = 30,
    validation_data = validation_generator,
    validation_steps = 50)

my_network_dropout.save('D:/2-AI读书学习笔记/0-python深度学习/0-model_repo/cats_and_dogs_small_my_network_with_dropout.h5')

import pickle
with open('D:/2-AI读书学习笔记/0-python深度学习/0-model_repo/ch5_2_network_cnn_dropout_trainHistoryDict.txt', 'wb') as file_pi:
    pickle.dump(history_droupout.history, file_pi)#将训练历史数据保存到指定文件中

Epoch 1/30
100/100 [==============================] - 67s 669ms/step - loss: 0.6924 - acc: 0.5180 - val_loss: 0.6818 - val_acc: 0.6140
Epoch 2/30
100/100 [==============================] - 68s 684ms/step - loss: 0.6766 - acc: 0.5730 - val_loss: 0.6723 - val_acc: 0.5670
Epoch 3/30
100/100 [==============================] - 66s 665ms/step - loss: 0.6571 - acc: 0.6165 - val_loss: 0.6401 - val_acc: 0.6390
Epoch 4/30
100/100 [==============================] - 67s 667ms/step - loss: 0.6341 - acc: 0.6395 - val_loss: 0.6413 - val_acc: 0.6310
Epoch 5/30
100/100 [==============================] - 70s 699ms/step - loss: 0.6044 - acc: 0.6700 - val_loss: 0.6104 - val_acc: 0.6700
Epoch 6/30
100/100 [==============================] - 98s 983ms/step - loss: 0.5864 - acc: 0.6935 - val_loss: 0.5916 - val_acc: 0.6970
Epoch 7/30
100/100 [==============================] - 91s 905ms/step - loss: 0.5465 - acc: 0.7225 - val_loss: 0.5683 - val_acc: 0.7050
Epoch 8/30
100/100 [==============================] - 7

### 1.3 评估dropout选项对模型在测试集分类准确率的影响

In [74]:
import os
from keras.preprocessing import image

classes = ['cats','dogs']
files_name = []
test_true_labels = []
test_pred_labels = []
test_pred_probs = []

for c in classes:
    test_class_dir = 'D:/2-AI读书学习笔记/99-dataset-lib/cat_dog/small_dataset_cat_dog/test/{}'.format(c)
    for f in os.listdir(test_class_dir):
        
        files_name.append(f)
        test_true_labels.append(f[0:3])
        
        img_path = os.path.join(test_class_dir, f)
        img = image.load_img(img_path, target_size = (150, 150))
        #模型预测
        pred = my_network_dropout.predict(preprocess_img(img))
        test_pred_probs.append(pred)
        
        pred_label = None
        if pred[0] > 0.5:
            pred_label = 'dog'
        else:
            pred_label = 'cat'
        test_pred_labels.append(pred_label)


import pandas as pd
df = pd.DataFrame(data={'file':files_name, 'true_label':test_true_labels,  'pred_label':test_pred_labels, 'pred_prob':test_pred_probs})


        
pred_correct_records = df[df['true_label'] == df['pred_label']].shape[0]
correct_rate = float(pred_correct_records)/float(1000)
correct_rate        

0.749

## 2 对照组之二：仅使用L1范数

### 2.1 按需建模，设置regularizers.l1(0.001)

In [59]:
from keras import models
from keras import layers
from keras import regularizers

#conv2D的filters从32增加到128
my_network_L1 = models.Sequential()
#32个filter
my_network_L1.add(layers.Conv2D(32,(3,3),activation = 'relu',input_shape = (150,150,3)))
my_network_L1.add(layers.MaxPooling2D((2,2)))
#64个filter
my_network_L1.add(layers.Conv2D(64,(3,3),activation = 'relu'))
my_network_L1.add(layers.MaxPooling2D((2,2)))
#128个filter
my_network_L1.add(layers.Conv2D(128,(3,3),activation = 'relu'))
my_network_L1.add(layers.MaxPooling2D((2,2)))
#128个filter
my_network_L1.add(layers.Conv2D(128,(3,3),activation = 'relu'))
my_network_L1.add(layers.MaxPooling2D((2,2)))

my_network_L1.add(layers.Flatten()) #将3D展平为1D
#为防止过拟合，使用dropout
#my_network_L1.add(layers.Dropout(0.5))
my_network_L1.add(layers.Dense(512, kernel_regularizer = regularizers.l1(0.001), activation = 'relu'))
my_network_L1.add(layers.Dense(1,activation='sigmoid'))#最后输出二元分类概率


from keras import optimizers
my_network_L1.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['acc'])

### 2.2 训练模型

In [60]:
history_L1 = my_network_L1.fit_generator(
    train_generator,
    steps_per_epoch = 100,
    epochs = 30,
    validation_data = validation_generator,
    validation_steps = 50)

my_network_L1.save('D:/2-AI读书学习笔记/0-python深度学习/0-model_repo/cats_and_dogs_small_my_network_with_L1.h5')
import pickle
with open('D:/2-AI读书学习笔记/0-python深度学习/0-model_repo/ch5_2_network_cnn_L1_trainHistoryDict.txt', 'wb') as file_pi:
    pickle.dump(history_L1.history, file_pi)#将训练历史数据保存到指定文件中

Epoch 1/30
100/100 [==============================] - 101s 1s/step - loss: 32.7300 - acc: 0.5090 - val_loss: 20.6112 - val_acc: 0.5000
Epoch 2/30
100/100 [==============================] - 100s 1s/step - loss: 12.3326 - acc: 0.5275 - val_loss: 5.6398 - val_acc: 0.5730
Epoch 3/30
100/100 [==============================] - 100s 995ms/step - loss: 2.5087 - acc: 0.5445 - val_loss: 0.9935 - val_acc: 0.5920
Epoch 4/30
100/100 [==============================] - 100s 1s/step - loss: 0.9218 - acc: 0.5505 - val_loss: 0.8914 - val_acc: 0.5200
Epoch 5/30
100/100 [==============================] - 100s 999ms/step - loss: 0.8742 - acc: 0.5580 - val_loss: 0.8545 - val_acc: 0.6010
Epoch 6/30
100/100 [==============================] - 100s 999ms/step - loss: 0.8605 - acc: 0.5660 - val_loss: 0.8498 - val_acc: 0.5990
Epoch 7/30
100/100 [==============================] - 101s 1s/step - loss: 0.8512 - acc: 0.5670 - val_loss: 0.8442 - val_acc: 0.5670
Epoch 8/30
100/100 [==============================] - 99s

### 2.3 评估L1选项对模型在测试集分类准确率的影响

In [75]:
import os
from keras.preprocessing import image

classes = ['cats','dogs']
files_name = []
test_true_labels = []
test_pred_labels = []
test_pred_probs = []

for c in classes:
    test_class_dir = 'D:/2-AI读书学习笔记/99-dataset-lib/cat_dog/small_dataset_cat_dog/test/{}'.format(c)
    for f in os.listdir(test_class_dir):
        
        files_name.append(f)
        test_true_labels.append(f[0:3])
        
        img_path = os.path.join(test_class_dir, f)
        img = image.load_img(img_path, target_size = (150, 150))
        #模型预测
        pred = my_network_L1.predict(preprocess_img(img))
        test_pred_probs.append(pred)
        
        pred_label = None
        if pred[0] > 0.5:
            pred_label = 'dog'
        else:
            pred_label = 'cat'
        test_pred_labels.append(pred_label)


import pandas as pd
df = pd.DataFrame(data={'file':files_name, 'true_label':test_true_labels,  'pred_label':test_pred_labels, 'pred_prob':test_pred_probs})


        
pred_correct_records = df[df['true_label'] == df['pred_label']].shape[0]
correct_rate = float(pred_correct_records)/float(1000)
correct_rate    

0.674

## 3 对照组之三：仅使用L1范数
### 3.1 按需建模，设置regularizers.l2(0.001)

In [62]:
from keras import models
from keras import layers
from keras import regularizers

#conv2D的filters从32增加到128
my_network_L2 = models.Sequential()
#32个filter
my_network_L2.add(layers.Conv2D(32,(3,3),activation = 'relu',input_shape = (150,150,3)))
my_network_L2.add(layers.MaxPooling2D((2,2)))
#64个filter
my_network_L2.add(layers.Conv2D(64,(3,3),activation = 'relu'))
my_network_L2.add(layers.MaxPooling2D((2,2)))
#128个filter
my_network_L2.add(layers.Conv2D(128,(3,3),activation = 'relu'))
my_network_L2.add(layers.MaxPooling2D((2,2)))
#128个filter
my_network_L2.add(layers.Conv2D(128,(3,3),activation = 'relu'))
my_network_L2.add(layers.MaxPooling2D((2,2)))

my_network_L2.add(layers.Flatten()) #将3D展平为1D
#为防止过拟合，使用dropout
#my_network_L1.add(layers.Dropout(0.5))
my_network_L2.add(layers.Dense(512, kernel_regularizer = regularizers.l2(0.001), activation = 'relu'))
my_network_L2.add(layers.Dense(1,activation='sigmoid'))#最后输出二元分类概率


from keras import optimizers
my_network_L2.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['acc'])

### 3.2 训练模型

In [63]:
history_L2 = my_network_L2.fit_generator(
    train_generator,
    steps_per_epoch = 100,
    epochs = 30,
    validation_data = validation_generator,
    validation_steps = 50)

my_network_L2.save('D:/2-AI读书学习笔记/0-python深度学习/0-model_repo/cats_and_dogs_small_my_network_with_L2.h5')
import pickle
with open('D:/2-AI读书学习笔记/0-python深度学习/0-model_repo/ch5_2_network_cnn_L2_trainHistoryDict.txt', 'wb') as file_pi:
    pickle.dump(history_L2.history, file_pi)#将训练历史数据保存到指定文件中

Epoch 1/30
100/100 [==============================] - 67s 674ms/step - loss: 1.3403 - acc: 0.5390 - val_loss: 1.1124 - val_acc: 0.5910
Epoch 2/30
100/100 [==============================] - 68s 683ms/step - loss: 0.9948 - acc: 0.5935 - val_loss: 0.8991 - val_acc: 0.6300
Epoch 3/30
100/100 [==============================] - 68s 685ms/step - loss: 0.8481 - acc: 0.6570 - val_loss: 0.8235 - val_acc: 0.6520
Epoch 4/30
100/100 [==============================] - 69s 685ms/step - loss: 0.7665 - acc: 0.7025 - val_loss: 0.7663 - val_acc: 0.6860
Epoch 5/30
100/100 [==============================] - 68s 677ms/step - loss: 0.7153 - acc: 0.7305 - val_loss: 0.8328 - val_acc: 0.6210
Epoch 6/30
100/100 [==============================] - 69s 686ms/step - loss: 0.6714 - acc: 0.7410 - val_loss: 0.7364 - val_acc: 0.6700
Epoch 7/30
100/100 [==============================] - 68s 680ms/step - loss: 0.6367 - acc: 0.7565 - val_loss: 0.6934 - val_acc: 0.7090
Epoch 8/30
100/100 [==============================] - 6

### 3.3 评估L2选项对模型在测试集分类准确率上的影响

In [76]:
import os
from keras.preprocessing import image

classes = ['cats','dogs']
files_name = []
test_true_labels = []
test_pred_labels = []
test_pred_probs = []

for c in classes:
    test_class_dir = 'D:/2-AI读书学习笔记/99-dataset-lib/cat_dog/small_dataset_cat_dog/test/{}'.format(c)
    for f in os.listdir(test_class_dir):
        
        files_name.append(f)
        test_true_labels.append(f[0:3])
        
        img_path = os.path.join(test_class_dir, f)
        img = image.load_img(img_path, target_size = (150, 150))
        #模型预测
        pred = my_network_L2.predict(preprocess_img(img))
        test_pred_probs.append(pred)
        
        pred_label = None
        if pred[0] > 0.5:
            pred_label = 'dog'
        else:
            pred_label = 'cat'
        test_pred_labels.append(pred_label)


import pandas as pd
df = pd.DataFrame(data={'file':files_name, 'true_label':test_true_labels,  'pred_label':test_pred_labels, 'pred_prob':test_pred_probs})


        
pred_correct_records = df[df['true_label'] == df['pred_label']].shape[0]
correct_rate = float(pred_correct_records)/float(1000)
correct_rate   

0.714

## 4 对照组之四：仅缩减网络结构

### 4.1 按需建模，卷积层由4层减少为3层

In [65]:
from keras import models
from keras import layers
from keras import regularizers

#conv2D的filters从32增加到128
my_network_simple = models.Sequential()
#32个filter
my_network_simple.add(layers.Conv2D(32,(3,3),activation = 'relu',input_shape = (150,150,3)))
my_network_simple.add(layers.MaxPooling2D((2,2)))
#64个filter
my_network_simple.add(layers.Conv2D(64,(3,3),activation = 'relu'))
my_network_simple.add(layers.MaxPooling2D((2,2)))
#128个filter
my_network_simple.add(layers.Conv2D(128,(3,3),activation = 'relu'))
my_network_simple.add(layers.MaxPooling2D((2,2)))
# #128个filter
# my_network_L2.add(layers.Conv2D(128,(3,3),activation = 'relu'))
# my_network_L2.add(layers.MaxPooling2D((2,2)))

my_network_simple.add(layers.Flatten()) #将3D展平为1D
#为防止过拟合，使用dropout
#my_network_L1.add(layers.Dropout(0.5))
my_network_simple.add(layers.Dense(512,activation = 'relu'))
my_network_simple.add(layers.Dense(1,activation='sigmoid'))#最后输出二元分类概率


from keras import optimizers
my_network_simple.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['acc'])

### 4.2 训练模型

In [66]:
history_simple = my_network_simple.fit_generator(
    train_generator,
    steps_per_epoch = 100,
    epochs = 30,
    validation_data = validation_generator,
    validation_steps = 50)

my_network_simple.save('D:/2-AI读书学习笔记/0-python深度学习/0-model_repo/cats_and_dogs_small_my_network_with_simple.h5')

import pickle
with open('D:/2-AI读书学习笔记/0-python深度学习/0-model_repo/ch5_2_network_cnn_simple_trainHistoryDict.txt', 'wb') as file_pi:
    pickle.dump(history_simple.history, file_pi)#将训练历史数据保存到指定文件中

Epoch 1/30
100/100 [==============================] - 77s 770ms/step - loss: 0.6835 - acc: 0.5740 - val_loss: 0.7233 - val_acc: 0.5540
Epoch 2/30
100/100 [==============================] - 77s 766ms/step - loss: 0.6044 - acc: 0.6730 - val_loss: 0.6188 - val_acc: 0.6490
Epoch 3/30
100/100 [==============================] - 77s 767ms/step - loss: 0.5344 - acc: 0.7335 - val_loss: 0.5982 - val_acc: 0.6780
Epoch 4/30
100/100 [==============================] - 77s 774ms/step - loss: 0.4931 - acc: 0.7650 - val_loss: 0.7499 - val_acc: 0.5970
Epoch 5/30
100/100 [==============================] - 76s 764ms/step - loss: 0.4521 - acc: 0.7830 - val_loss: 0.6823 - val_acc: 0.6570
Epoch 6/30
100/100 [==============================] - 78s 784ms/step - loss: 0.4078 - acc: 0.8175 - val_loss: 0.6023 - val_acc: 0.6890
Epoch 7/30
100/100 [==============================] - 77s 774ms/step - loss: 0.3681 - acc: 0.8420 - val_loss: 0.6210 - val_acc: 0.6820
Epoch 8/30
100/100 [==============================] - 7

### 4.3 评估缩减CNN网络规模对模型在测试集分类准确率上的影响

In [77]:
import os
from keras.preprocessing import image

classes = ['cats','dogs']
files_name = []
test_true_labels = []
test_pred_labels = []
test_pred_probs = []

for c in classes:
    test_class_dir = 'D:/2-AI读书学习笔记/99-dataset-lib/cat_dog/small_dataset_cat_dog/test/{}'.format(c)
    for f in os.listdir(test_class_dir):
        
        files_name.append(f)
        test_true_labels.append(f[0:3])
        
        img_path = os.path.join(test_class_dir, f)
        img = image.load_img(img_path, target_size = (150, 150))
        #模型预测
        pred = my_network_simple.predict(preprocess_img(img))
        test_pred_probs.append(pred)
        
        pred_label = None
        if pred[0] > 0.5:
            pred_label = 'dog'
        else:
            pred_label = 'cat'
        test_pred_labels.append(pred_label)


import pandas as pd
df = pd.DataFrame(data={'file':files_name, 'true_label':test_true_labels,  'pred_label':test_pred_labels, 'pred_prob':test_pred_probs})


        
pred_correct_records = df[df['true_label'] == df['pred_label']].shape[0]
correct_rate = float(pred_correct_records)/float(1000)
correct_rate   

0.711

### 5 对照组之五：同时使用dropout与数据增强技术

### 5.1 按需建模，droupout设置为0.5；同时设置带数据增强的图片生成器实例

In [ ]:
train_datagen_with_da = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest')

train_generator_with_da = train_datagen_with_da.flow_from_directory(
    train_dir,
    target_size = (150,150),
    batch_size = 20,
    class_mode = 'binary')

In [13]:
from keras import models
from keras import layers
from keras import regularizers

#conv2D的filters从32增加到128
my_network_droupout_da = models.Sequential()
#32个filter
my_network_droupout_da.add(layers.Conv2D(32,(3,3),activation = 'relu',input_shape = (150,150,3)))
my_network_droupout_da.add(layers.MaxPooling2D((2,2)))
#64个filter
my_network_droupout_da.add(layers.Conv2D(64,(3,3),activation = 'relu'))
my_network_droupout_da.add(layers.MaxPooling2D((2,2)))
#128个filter
my_network_droupout_da.add(layers.Conv2D(128,(3,3),activation = 'relu'))
my_network_droupout_da.add(layers.MaxPooling2D((2,2)))
# #128个filter
# my_network_L2.add(layers.Conv2D(128,(3,3),activation = 'relu'))
# my_network_L2.add(layers.MaxPooling2D((2,2)))

my_network_droupout_da.add(layers.Flatten()) #将3D展平为1D
#为防止过拟合，使用dropout
my_network_droupout_da.add(layers.Dropout(0.5))
my_network_droupout_da.add(layers.Dense(512, activation = 'relu'))
my_network_droupout_da.add(layers.Dense(1,activation='sigmoid'))#最后输出二元分类概率


from keras import optimizers
my_network_droupout_da.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['acc'])

### 5.2 训练模型

In [14]:
history_droupout_da = my_network_droupout_da.fit_generator(
    train_generator_with_da,
    steps_per_epoch = 100,
    epochs = 30,
    validation_data = validation_generator,
    validation_steps = 50)

my_network_droupout_da.save('D:/2-AI读书学习笔记/0-python深度学习/0-model_repo/cats_and_dogs_small_my_network_with_dropout_da.h5')

import pickle
with open('D:/2-AI读书学习笔记/0-python深度学习/0-model_repo/ch5_2_network_cnn_droupout_da_trainHistoryDict.txt', 'wb') as file_pi:
    pickle.dump(history_droupout_da.history, file_pi)#将训练历史数据保存到指定文件中

Epoch 1/30
100/100 [==============================] - 76s 763ms/step - loss: 0.7043 - acc: 0.5330 - val_loss: 0.6821 - val_acc: 0.5870
Epoch 2/30
100/100 [==============================] - 78s 778ms/step - loss: 0.6837 - acc: 0.5495 - val_loss: 0.6640 - val_acc: 0.5860
Epoch 3/30
100/100 [==============================] - 76s 758ms/step - loss: 0.6699 - acc: 0.5870 - val_loss: 0.6416 - val_acc: 0.6360
Epoch 4/30
100/100 [==============================] - 77s 769ms/step - loss: 0.6579 - acc: 0.6180 - val_loss: 0.6650 - val_acc: 0.5730
Epoch 5/30
100/100 [==============================] - 77s 769ms/step - loss: 0.6336 - acc: 0.6400 - val_loss: 0.6001 - val_acc: 0.6810
Epoch 6/30
100/100 [==============================] - 77s 766ms/step - loss: 0.6275 - acc: 0.6450 - val_loss: 0.6541 - val_acc: 0.5950
Epoch 7/30
100/100 [==============================] - 75s 749ms/step - loss: 0.6111 - acc: 0.6620 - val_loss: 0.6466 - val_acc: 0.6160
Epoch 8/30
100/100 [==============================] - 7

### 5.3 评估dropout与数据增强两个选项复合使用时，对模型在测试集的分类准确率的影响

In [17]:
import os
from keras.preprocessing import image

classes = ['cats','dogs']
files_name = []
test_true_labels = []
test_pred_labels = []
test_pred_probs = []

for c in classes:
    test_class_dir = 'D:/2-AI读书学习笔记/99-dataset-lib/cat_dog/small_dataset_cat_dog/test/{}'.format(c)
    for f in os.listdir(test_class_dir):
        
        files_name.append(f)
        test_true_labels.append(f[0:3])
        
        img_path = os.path.join(test_class_dir, f)
        img = image.load_img(img_path, target_size = (150, 150))
        pred = my_network_droupout_da.predict(preprocess_img(img))
        test_pred_probs.append(pred)
        
        pred_label = None
        if pred[0] > 0.5:
            pred_label = 'dog'
        else:
            pred_label = 'cat'
        test_pred_labels.append(pred_label) 
        

import pandas as pd
df = pd.DataFrame(data={'file':files_name, 'true_label':test_true_labels,  'pred_label':test_pred_labels, 'pred_prob':test_pred_probs})


        
pred_correct_records = df[df['true_label'] == df['pred_label']].shape[0]
correct_rate = float(pred_correct_records)/float(1000)
correct_rate 

0.749